# Import Library

In [50]:
import json
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from google.colab import drive
# drive.mount('/content/drive/')
# Path = '/content/drive/MyDrive/lab_assignment_2/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


# Read & Analyze Data

In [51]:
file_path = 'training_data.txt'

column_names = ["features1", "features2", "labels3"]
df = pd.read_csv(file_path, names=column_names)

df.head()

,features1,features2,labels3
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [52]:
df.describe()

,features1,features2,labels3
count,118.000000,118.000000,118.000000
mean,0.054779,0.183102,0.491525
std,0.496654,0.519743,0.502060
min,-0.830070,-0.769740,0.000000
25%,-0.372120,-0.254385,0.000000
50%,-0.006336,0.213455,0.000000
75%,0.478970,0.646563,1.000000
max,1.070900,1.108900,1.000000


# Helper Functions

In [53]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def cost_reg(theta, X, y, lambda_):
    m = y.size
    h = sigmoid(X @ theta)
    t1 = -y.T @ np.log(h)
    t2 = (1 - y).T @ np.log(1 - h)
    cost = (1 / m) * np.sum(t1 - t2)
    reg_term = (lambda_ / (2*m)) * np.sum(theta[1:] ** 2)
    return cost + reg_term

def gradient_reg(theta, X, y, lambda_):
    m = y.size
    h = sigmoid(X @ theta)
    grad = (1 / m) * (X.T @ (h - y))
    reg_term = (lambda_ / m) * theta
    reg_term[0] = 0
    return grad + reg_term

def gradient_descent(theta, X, y, alpha, num_iters, lambda_):
    for i in range(num_iters):
        theta = theta - alpha * gradient_reg(theta, X, y, lambda_)
    return theta

def predict(theta, X):
    prob = sigmoid(X @ theta)
    return (prob >= 0.5).astype(int)


# Map Function

In [54]:
def map_feature(x1, x2):
#   x1, x2 type: numpy array
#   Returns a new feature array with more features, comprising of
#   x1, x2, x1.^2, x2.^2, x1*x2, x1*x2.^2, etc.

    degree = 6
    out = np.ones([len(x1), int((degree + 1) * (degree + 2) / 2)])
    idx = 1

    for i in range(1, degree + 1):
        for j in range(0, i + 1):
            a1 = x1 ** (i - j)
            a2 = x2 ** j
            out[:, idx] = a1 * a2
            idx += 1

    return out

# Main Function

In [55]:
# Import json
with open('config.json',) as f:
    configs = json.load(f)

# Import data
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()
X_mapped = map_feature(X[:, 0], X[:, 1])
theta = np.zeros(X_mapped.shape[1])

# gradient decent
theta = gradient_descent(theta, X_mapped, y, alpha=configs["Alpha"], num_iters=configs["NumIter"], lambda_=configs["Lambda"])
y_pred = predict(theta, X_mapped)

# redictions
accuracy = accuracy_score(y, y_pred)
precision = precision_score(y, y_pred)
recall = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Save model
with open('model.json', 'w') as f:
    model = { 'theta: ': theta.tolist() }
    json.dump(model, f, indent=4)

# Save evaluations
with open('classification_report.json', 'w') as f:
    score = {
        'Accuracy: ': accuracy,
        'Precision: ': precision,
        'Recall: ': recall,
        'F1 Score: ': f1
    }
    json.dump(score, f, indent=4)

Accuracy: 0.8305084745762712
Precision: 0.7794117647058824
Recall: 0.9137931034482759
F1 Score: 0.8412698412698414
